In [20]:
"""
References:
https://arxiv.org/pdf/1512.03385.pdf
https://arxiv.org/pdf/1604.04112v4.pdf
https://github.com/yu4u/cutout-random-erasing
"""
import keras
import numpy as np
from keras.layers import Add, Dense, Conv2D, BatchNormalization
from keras.layers import Activation, AveragePooling2D, Input, Flatten
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.datasets import cifar10
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.models import Model

In [21]:
batch_size = 128
epochs = 180
n_classes = 10
learning_rate = 0.1

In [22]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

In [23]:
class Resnet:
    def __init__(self, size=44, stacks=3, starting_filter=16):
        self.size = size
        self.stacks = stacks
        self.starting_filter = starting_filter
        self.residual_blocks = (size - 2) // 6
        
    def get_model(self, input_shape=(32, 32, 3), n_classes=10):
        n_filters = self.starting_filter

        inputs = Input(shape=input_shape)
        network = self.layer(inputs, n_filters)
        network = self.stack(network, n_filters, True)

        for _ in range(self.stacks - 1):
            n_filters *= 2
            network = self.stack(network, n_filters)

        network = Activation('elu')(network)
        network = AveragePooling2D(pool_size=network.shape[1])(network)
        network = Flatten()(network)
        outputs = Dense(n_classes, activation='softmax', 
                        kernel_initializer='he_normal')(network)

        model = Model(inputs=inputs, outputs=outputs)

        return model
    
    def stack(self, inputs, n_filters, first_stack=False):
        stack = inputs

        if first_stack:
            stack = self.identity_block(stack, n_filters)
        else:
            stack = self.convolution_block(stack, n_filters)

        for _ in range(self.residual_blocks - 1):
            stack = self.identity_block(stack, n_filters)

        return stack
    
    def identity_block(self, inputs, n_filters):
        shortcut = inputs

        block = self.layer(inputs, n_filters, normalize_batch=False)
        block = self.layer(block, n_filters, activation=None)

        block = Add()([shortcut, block])

        return block

    def convolution_block(self, inputs, n_filters, strides=2):
        shortcut = inputs

        block = self.layer(inputs, n_filters, strides=strides,
                           normalize_batch=False)
        block = self.layer(block, n_filters, activation=None)

        shortcut = self.layer(shortcut, n_filters,
                              kernel_size=1, strides=strides,
                              activation=None)

        block = Add()([shortcut, block])

        return block
    
    def layer(self, inputs, n_filters, kernel_size=3,
              strides=1, activation='elu', normalize_batch=True):
    
        convolution = Conv2D(n_filters, kernel_size=kernel_size,
                             strides=strides, padding='same',
                             kernel_initializer="he_normal",
                             kernel_regularizer=l2(1e-4))

        x = convolution(inputs)

        if normalize_batch:
            x = BatchNormalization()(x)

        if activation is not None:
            x = Activation(activation)(x)

        return x
    
def learning_rate_schedule(epoch):
    new_learning_rate = learning_rate

    if epoch <= 90:
        pass
    elif epoch > 90 and epoch <= 140:
        new_learning_rate = learning_rate * 0.1
    else:
        new_learning_rate = learning_rate * 0.01
        
    print('Learning rate:', new_learning_rate)
    
    return new_learning_rate

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        if input_img.ndim == 3:
            img_h, img_w, img_c = input_img.shape
        elif input_img.ndim == 2:
            img_h, img_w = input_img.shape

        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            if input_img.ndim == 3:
                c = np.random.uniform(v_l, v_h, (h, w, img_c))
            if input_img.ndim == 2:
                c = np.random.uniform(v_l, v_h, (h, w))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w] = c

        return input_img

    return eraser

In [24]:
resnet = Resnet()

model = resnet.get_model()

optimizer = SGD(learning_rate=learning_rate, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_135 (Conv2D)             (None, 32, 32, 16)   448         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_72 (BatchNo (None, 32, 32, 16)   64          conv2d_135[0][0]                 
__________________________________________________________________________________________________
activation_69 (Activation)      (None, 32, 32, 16)   0           batch_normalization_72[0][0]     
____________________________________________________________________________________________

In [25]:
lr_scheduler = LearningRateScheduler(learning_rate_schedule)
callbacks = [lr_scheduler]

datagen = ImageDataGenerator(width_shift_range=4,
                             height_shift_range=4,
                             horizontal_flip=True,
                             preprocessing_function=get_random_eraser(p=1, pixel_level=True))
datagen.fit(x_train)

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
          validation_data=(x_test, y_test),
          epochs=epochs, workers=4,
          callbacks=callbacks)

model.save('model.h5')

Epoch 1/180
Learning rate: 0.1
391/391 [==============================] - 30s 66ms/step - loss: 2.1054 - accuracy: 0.3555 - val_loss: 1.8543 - val_accuracy: 0.4416
Epoch 2/180
Learning rate: 0.1
391/391 [==============================] - 27s 69ms/step - loss: 1.6766 - accuracy: 0.5070 - val_loss: 1.7795 - val_accuracy: 0.4819
Epoch 3/180
Learning rate: 0.1
391/391 [==============================] - 27s 68ms/step - loss: 1.4538 - accuracy: 0.5816 - val_loss: 1.2391 - val_accuracy: 0.6563
Epoch 4/180
Learning rate: 0.1
391/391 [==============================] - 28s 70ms/step - loss: 1.3039 - accuracy: 0.6334 - val_loss: 1.4266 - val_accuracy: 0.5979
Epoch 5/180
Learning rate: 0.1
391/391 [==============================] - 26s 67ms/step - loss: 1.1892 - accuracy: 0.6694 - val_loss: 1.1024 - val_accuracy: 0.7098
Epoch 6/180
Learning rate: 0.1
391/391 [==============================] - 27s 68ms/step - loss: 1.0812 - accuracy: 0.7051 - val_loss: 1.1525 - val_accuracy: 0.6953
Epoch 7/180
Lear

391/391 [==============================] - 27s 68ms/step - loss: 0.6497 - accuracy: 0.8650 - val_loss: 0.6287 - val_accuracy: 0.8777
Epoch 51/180
Learning rate: 0.1
391/391 [==============================] - 27s 69ms/step - loss: 0.6388 - accuracy: 0.8691 - val_loss: 0.6443 - val_accuracy: 0.8691
Epoch 52/180
Learning rate: 0.1
391/391 [==============================] - 27s 67ms/step - loss: 0.6429 - accuracy: 0.8679 - val_loss: 0.6347 - val_accuracy: 0.8785
Epoch 53/180
Learning rate: 0.1
391/391 [==============================] - 27s 69ms/step - loss: 0.6389 - accuracy: 0.8683 - val_loss: 0.6642 - val_accuracy: 0.8731
Epoch 54/180
Learning rate: 0.1
391/391 [==============================] - 27s 68ms/step - loss: 0.6421 - accuracy: 0.8699 - val_loss: 0.6511 - val_accuracy: 0.8697
Epoch 55/180
Learning rate: 0.1
391/391 [==============================] - 28s 71ms/step - loss: 0.6403 - accuracy: 0.8704 - val_loss: 0.6784 - val_accuracy: 0.8649
Epoch 56/180
Learning rate: 0.1
391/391 [=

391/391 [==============================] - 28s 70ms/step - loss: 0.4056 - accuracy: 0.9470 - val_loss: 0.4430 - val_accuracy: 0.9363
Epoch 100/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 29s 73ms/step - loss: 0.3960 - accuracy: 0.9491 - val_loss: 0.4389 - val_accuracy: 0.9376
Epoch 101/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 28s 72ms/step - loss: 0.3880 - accuracy: 0.9497 - val_loss: 0.4315 - val_accuracy: 0.9386
Epoch 102/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 28s 72ms/step - loss: 0.3843 - accuracy: 0.9510 - val_loss: 0.4278 - val_accuracy: 0.9397
Epoch 103/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 28s 72ms/step - loss: 0.3770 - accuracy: 0.9514 - val_loss: 0.4297 - val_accuracy: 0.9393
Epoch 104/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 28s 71ms/step - loss: 0.3698 - accuracy: 0

391/391 [==============================] - 29s 73ms/step - loss: 0.2311 - accuracy: 0.9721 - val_loss: 0.3370 - val_accuracy: 0.9459
Epoch 145/180
Learning rate: 0.001
391/391 [==============================] - 28s 70ms/step - loss: 0.2247 - accuracy: 0.9748 - val_loss: 0.3367 - val_accuracy: 0.9456
Epoch 146/180
Learning rate: 0.001
391/391 [==============================] - 28s 70ms/step - loss: 0.2281 - accuracy: 0.9729 - val_loss: 0.3351 - val_accuracy: 0.9453
Epoch 147/180
Learning rate: 0.001
391/391 [==============================] - 29s 75ms/step - loss: 0.2253 - accuracy: 0.9746 - val_loss: 0.3362 - val_accuracy: 0.9468
Epoch 148/180
Learning rate: 0.001
391/391 [==============================] - 28s 72ms/step - loss: 0.2237 - accuracy: 0.9745 - val_loss: 0.3355 - val_accuracy: 0.9463
Epoch 149/180
Learning rate: 0.001
391/391 [==============================] - 28s 71ms/step - loss: 0.2211 - accuracy: 0.9750 - val_loss: 0.3353 - val_accuracy: 0.9462
Epoch 150/180
Learning rate